In [8]:
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.2 MB/s eta 0:00:00


In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

To preprocess our data, we will need a tokenizer:

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Next we will generate a small subset of the training and validation set, to enable faster training:

In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

First, let’s define our model:

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


It will warn about some pre-trained weights not being used and some weights being randomly initialized. That’s because we are throwing away the pretraining head of the BERT model to replace it with a classification head that is randomly initialized. We will fine-tune this model on our task, transferring the knowledge of the pre-trained model to it (which is why it is called transfer learning).

Then, to define our Trainer, we will need to instantiate a TrainingArguments. This class contains all the hyperparameters we can tune for the Trainer or the flags to activate the different training options it supports. Let’s begin by using all the defaults. The only thing we then have to provide is a directory in which the checkpoints will be saved:

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16)

In [11]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,

)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.320310,0.874000
2,No log,0.525860,0.873000
3,No log,0.619589,0.874000


TrainOutput(global_step=189, training_loss=0.15365995174993283, metrics={'train_runtime': 387.2058, 'train_samples_per_second': 7.748, 'train_steps_per_second': 0.488, 'total_flos': 789333166080000.0, 'train_loss': 0.15365995174993283, 'epoch': 3.0})

In [14]:
full_eval_dataset.select([0]).to_dict()["text"]

['I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they hav

In [15]:
trainer.predict(full_eval_dataset.select([0]))

PredictionOutput(predictions=array([[ 2.770835 , -3.4391346]], dtype=float32), label_ids=array([0]), metrics={'test_loss': 0.0020072567276656628, 'test_accuracy': 1.0, 'test_runtime': 0.09, 'test_samples_per_second': 11.106, 'test_steps_per_second': 11.106})